<a href="https://colab.research.google.com/github/LiamTodd/inclusiviser-mvp/blob/main/Alpaca_LLM_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Alpaca LLM Fast API

Set the runtime to GPU

##Setup
This may take a while to run, give it around 10-15 minutes.

In [ ]:
!pip install -q datasets loralib sentencepiece
!pip uninstall transformers
!pip install -q git+https://github.com/zphang/transformers@c3dc391
!pip -q install git+https://github.com/huggingface/peft.git
!pip -q install bitsandbytes

from peft import PeftModel
from transformers import LLaMATokenizer, LLaMAForCausalLM, GenerationConfig
import textwrap

tokenizer = LLaMATokenizer.from_pretrained("decapoda-research/llama-7b-hf")

model = LLaMAForCausalLM.from_pretrained(
    "decapoda-research/llama-7b-hf",
    load_in_8bit=True,
    device_map="auto",
)
model = PeftModel.from_pretrained(model, "samwit/alpaca7B-lora")


def alpaca_talk(text, temperature=0.5, top_p=0.95, repetition_penalty=1.2, max_new_tokens=1024):
    inputs = tokenizer(
        text,
        return_tensors="pt",
    )
    input_ids = inputs["input_ids"].cuda()

    generation_config = GenerationConfig(
        temperature=temperature,
        top_p=top_p,
        repetition_penalty=repetition_penalty,
    )
    generation_output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=max_new_tokens,
    )
    for s in generation_output.sequences:
        return tokenizer.decode(s)

import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install fastapi nest-asyncio pyngrok uvicorn

##FastAPI
Executing this will generate a temporary ngrok url which can be used as the root of the API.

In [ ]:
from fastapi import FastAPI
import nest_asyncio
from pyngrok import ngrok
import uvicorn

app = FastAPI()

@app.get('/')
async def generate_response(input):
  output = alpaca_talk(input)
  print(output)
  return {"data": output}

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)